# Problema 1

## Descripción:

En este problema, se presenta un conjunto de datos que contiene información sobre el rendimiento académico de estudiantes universitarios, así como diversos factores que podrían influir en él. El objetivo es construir un modelo de regresión utilizando redes neuronales para predecir el índice de rendimiento académico de los estudiantes basado en las características proporcionadas.

## Dataset:

El dataset proporcionado incluye las siguientes variables para cada estudiante:

- Hours Studied: El número total de horas dedicadas al estudio por cada estudiante.
- Previous Scores: Las puntuaciones obtenidas por los estudiantes en exámenes previos.
- Extracurricular Activities: Si el estudiante participa en actividades extracurriculares (Sí o No).
- Sleep Hours: El número promedio de horas de sueño que el estudiante tuvo por día.
- Sample Question Papers Practiced: El número de cuestionarios de muestra que el estudiante practicó.

Además, el dataset incluye la variable objetivo:

- Performance Index: Un índice que representa el rendimiento académico general de cada estudiante, redondeado al entero más cercano. Este índice varía de 10 a 100, donde valores más altos indican un mejor rendimiento.

## Objetivo:

Utilizando el dataset proporcionado, el objetivo es construir un modelo de regresión utilizando redes neuronales que pueda predecir con precisión el índice de rendimiento académico de los estudiantes. Se debe entrenar y evaluar el modelo utilizando técnicas adecuadas de validación y métricas de evaluación de regresión.

## Entrega:

La entrega debe incluir:
- Código fuente de la solución implementada en Google Colab, que incluya:
- Análisis previo y preprocesamiento del set de datos.
- Definición y entrenamiento del modelo.
- Resultados de la evaluación del modelo, incluyendo métricas de desempeño y visualizaciones relevantes.

Nota: el código debe estar debidamente documentado con comentarios explicativos para que el trabajo sea fácilmente comprensible para otros revisores.

In [23]:
import zipfile
import os

In [24]:
student_performance = '1mfpXVLqDJah-sO0CF29LjKUz5NtKjZqc'

!gdown $student_performance

with zipfile.ZipFile('TP1-1-student_performance.zip', 'r') as zip_ref:
    if not os.path.exists('/content/'):
        os.makedirs('/content/')

    zip_ref.extractall('/content/')

!rm 'TP1-1-student_performance.zip'

Downloading...
From: https://drive.google.com/uc?id=1mfpXVLqDJah-sO0CF29LjKUz5NtKjZqc
To: /content/TP1-1-student_performance.zip
100% 49.7k/49.7k [00:00<00:00, 107MB/s]


In [25]:
import pandas as pd

In [26]:
df = pd.read_csv('/content/Student_Performance.csv')
df.head()

,Hours Studied,Previous Scores,Extracurricular Activities,Sleep Hours,Sample Question Papers Practiced,Performance Index
0,7,99,Yes,9,1,91.0
1,4,82,No,4,2,65.0
2,8,51,Yes,7,2,45.0
3,5,52,Yes,5,2,36.0
4,7,75,No,8,5,66.0


In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 6 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Hours Studied                     10000 non-null  int64  
 1   Previous Scores                   10000 non-null  int64  
 2   Extracurricular Activities        10000 non-null  object 
 3   Sleep Hours                       10000 non-null  int64  
 4   Sample Question Papers Practiced  10000 non-null  int64  
 5   Performance Index                 10000 non-null  float64
dtypes: float64(1), int64(4), object(1)
memory usage: 468.9+ KB


In [28]:
df.isnull().sum()

Hours Studied                       0
Previous Scores                     0
Extracurricular Activities          0
Sleep Hours                         0
Sample Question Papers Practiced    0
Performance Index                   0
dtype: int64

In [29]:
map_bools = lambda x: 1 if x == 'Yes' else 0

df["Extracurricular Activities"] = df["Extracurricular Activities"].apply(map_bools)

In [30]:
display(df.head(3))
print('\n')
display(df.info())

,Hours Studied,Previous Scores,Extracurricular Activities,Sleep Hours,Sample Question Papers Practiced,Performance Index
0,7,99,1,9,1,91.0
1,4,82,0,4,2,65.0
2,8,51,1,7,2,45.0




<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 6 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Hours Studied                     10000 non-null  int64  
 1   Previous Scores                   10000 non-null  int64  
 2   Extracurricular Activities        10000 non-null  int64  
 3   Sleep Hours                       10000 non-null  int64  
 4   Sample Question Papers Practiced  10000 non-null  int64  
 5   Performance Index                 10000 non-null  float64
dtypes: float64(1), int64(5)
memory usage: 468.9 KB


None

In [31]:
df.columns = df.columns.str.lower().str.replace(' ', '_')

In [32]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input

In [33]:
# Configurar para que TensorFlow utilice la GPU por defecto
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        print(e)

1 Physical GPUs, 1 Logical GPUs


In [34]:
from sklearn.model_selection import train_test_split

df_train, df_val = train_test_split(df, train_size=0.7, random_state=1)

In [36]:
# For train data.
X_train = df_train.drop('performance_index', axis=1)
y_train = df_train['performance_index']

# For validation data.
X_val = df_val.drop('performance_index', axis=1)
y_val = df_val['performance_index']

In [37]:
y_train.head()

2228    66.0
5910    52.0
1950    30.0
2119    83.0
5947    78.0
Name: performance_index, dtype: float64

In [62]:
from sklearn.preprocessing import StandardScaler

# Standardize the training data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

# Standardize the target column
y_train_scaled = scaler.fit_transform(np.array(y_train).reshape(-1, 1))
y_val_scaled = scaler.transform(np.array(y_val).reshape(-1, 1))

In [63]:
# Create the model
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(X_train_scaled.shape[1],)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='mse', metrics=['mse'])

# Train the model
model.fit(X_train_scaled, y_train_scaled)

# Evaluate the model
loss, mse = model.evaluate(X_val_scaled, y_val_scaled)

# Predict the target values for the val set
y_pred_scaled = model.predict(X_val_scaled)

# Inverse transform the predictions and target values
y_pred = scaler.inverse_transform(y_pred_scaled)
y_val = scaler.inverse_transform(y_val_scaled)

94/94 [==============================] - 0s 2ms/step


In [64]:
# Calculate the R² score
from sklearn.metrics import r2_score
r2 = r2_score(y_val, y_pred)

# Print the results
print('Loss:', loss)
print('Mean squared error:', mse)
print('R²:', r2)

Loss: 0.555341362953186
Mean squared error: 0.555341362953186
R²: 0.4325686116399493
